# Regresion Logistica: Deteccion de SPAM

En este ejercico se muestran los fundamentos de la regresion logistica, planteando uno de los primeros problemas que fueron solucionados mediante el uso ded tecnicas de Machine Learning: La deteccion de SPAM


##  Enunciado del ejercicio.
Se propone la construccion de un sistema de aprendizaje automatico capaz de predecir si un correo determinado se corresponde con un correo SPAM o no, para ello se utilizara el siguente DatSet:

##### [2007_TE _Public_Spam_Corpus (https://plg.uwaterloo.ca/~gvcormac/treccorpus07/)]
The corpus trec07p contains 75,419 messages:

    25220 ham
    50199 spam

These messages constitute all the messages delivered to a particular
server between these dates:

    Sun, 8 Apr 2007 13:07:21 -0400
    Fri, 6 Jul 2007 07:04:53 -0400

In [1]:
# En esta clase se facilita el procesamiento de correos electronicos 

# que poseen codigo html.
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return ''.join(self.fed)

In [2]:
# Esta funcion se encarga de eliminar los tags HTML
# que se encunetren en el texto de los correos electronicos
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
# Ejemplo de eliminacion de los tads HTML de un texto
t = '<tr><td align="left"><ahref="../../issues/51/16.html#article">Phrack world News </a><td>'
strip_tags(t)

'Phrack world News '

Ademas de eliminar los posiblrs tags html que se encuentran en el correo electronico deben realizarse otras acciones para evitar que los mensajes contengan ruido inecesario. Entre ellas se encuentra la eliminacion de signos de puntuacion, eliminancion de los posibles campos de correo electronico que no sean relevantes o eliminacion de los afijos de una palabra manteniendo unicamente la raiz de la misma(stemming). La clase que se muestra a continuacion realiza estas transformaciones.

In [4]:
import email
import string
import nltk

class Parser:
    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)
    def parse(self, email_path):
        """Parse an Email"""
        with open(email_path, errors = 'ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)
    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject']) if msg ['Subject'] else []
        body = self.get_email_body(msg.get_payload(),
                                    msg.get_content_type())
        content_type = msg.get_content_type()
        # Return the contentm of the email
        return {"subject": subject,
               "body": body,
               "content_type":content_type}
    def get_email_body(self, payload, content_type):
        """Extract the body of the email."""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                           p.get_content_type())
        return body
    def tokenize(self, text):
        """Transform a text string in tokens. Perform two main actions, clean the puntuaction symbols and do stemming of the text"""
        for c in self.punctuation:
            text = text.replace(c, "")
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        tokens = list(filter(None, text.split(" ")))
        # Stemming of the tokens
        return [self.stemmer.stem(w) for w in tokens if w not in self.stopwords]

Lectura de un correo en formato .raw

In [5]:
inmail = open("/home/lia/Escritorio/api/archivos/datasets/datasets/trec07p/data/inmail.1").read()
print(inmail)

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="

##### Parsing del correo electronico

In [6]:
p = Parser()
p.parse("/home/lia/Escritorio/api/archivos/datasets/datasets/trec07p/data/inmail.1")

{'subject': ['gener', 'ciali', 'brand', 'qualiti'],
 'body': ['do',
  'feel',
  'pressur',
  'perform',
  'rise',
  'occas',
  'tri',
  'viagra',
  'anxieti',
  'thing',
  'past',
  'back',
  'old',
  'self'],
 'content_type': 'multipart/alternative'}

##### Lectura del índice 
Estas funciones complementarias se encargan de cargar en memoria
la ruta de cada correo electronico y su etiqueta correspondiente.
{SPAM, ham}

In [7]:
index = open("/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/full/index").readlines()
index

['spam ../data/inmail.1\n',
 'ham ../data/inmail.2\n',
 'spam ../data/inmail.3\n',
 'spam ../data/inmail.4\n',
 'spam ../data/inmail.5\n',
 'spam ../data/inmail.6\n',
 'spam ../data/inmail.7\n',
 'spam ../data/inmail.8\n',
 'spam ../data/inmail.9\n',
 'ham ../data/inmail.10\n',
 'spam ../data/inmail.11\n',
 'spam ../data/inmail.12\n',
 'spam ../data/inmail.13\n',
 'spam ../data/inmail.14\n',
 'spam ../data/inmail.15\n',
 'spam ../data/inmail.16\n',
 'spam ../data/inmail.17\n',
 'spam ../data/inmail.18\n',
 'spam ../data/inmail.19\n',
 'ham ../data/inmail.20\n',
 'ham ../data/inmail.21\n',
 'spam ../data/inmail.22\n',
 'spam ../data/inmail.23\n',
 'spam ../data/inmail.24\n',
 'spam ../data/inmail.25\n',
 'spam ../data/inmail.26\n',
 'spam ../data/inmail.27\n',
 'spam ../data/inmail.28\n',
 'ham ../data/inmail.29\n',
 'spam ../data/inmail.30\n',
 'ham ../data/inmail.31\n',
 'spam ../data/inmail.32\n',
 'spam ../data/inmail.33\n',
 'ham ../data/inmail.34\n',
 'spam ../data/inmail.35\n',
 

In [8]:
import os 

DATASET_PATH = "/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p"

def parse_index(path_to_index, n_elements):
    ret_indexes = []
    index = open(path_to_index).readlines()
    for i in range(n_elements):
        mail = index[i].split(" ../")
        label = mail[0]
        path = mail[1][:-1]
        ret_indexes.append({"label": label, "email_path": os.path.join(DATASET_PATH, path)})
    return ret_indexes
        
    

In [9]:
def parse_email(index):
    p = Parser()
    pmail = p.parse(index["email_path"])
    return pmail, index["label"]

In [10]:
indexes = parse_index("/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/full/index", 10)
indexes

[{'label': 'spam',
  'email_path': '/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/data/inmail.1'},
 {'label': 'ham',
  'email_path': '/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/data/inmail.2'},
 {'label': 'spam',
  'email_path': '/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/data/inmail.3'},
 {'label': 'spam',
  'email_path': '/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/data/inmail.4'},
 {'label': 'spam',
  'email_path': '/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/data/inmail.5'},
 {'label': 'spam',
  'email_path': '/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/data/inmail.6'},
 {'label': 'spam',
  'email_path': '/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/data/inmail.7'},
 {'label': 'spam',
  'email_path': '/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/data/inmail.8'},
 {'label': 'spam',
  'email_path': '/home/lia/Escritorio/api/archivos/dat

# Prepocesamiento del DataSet.

Con las funciones presentadas anteriormente se permite la lectura de los correos electrónicos de manera programática y el 
procesamiento de los mismos para eliminar aquellos componentes que no resultan de utilidad para la detección de correos 
se SPAM. Sin embargo, cada uno de los correos sigue estando representado por un diccionario de Python con una serie de palabras.

In [11]:
# Cargar el índice y las etiquetas en memoria.
index = parse_index("/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/full/index", 1)

In [12]:
# Leemos el primer correo
import os

open(index[0]["email_path"]).read()

'From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007\nReturn-Path: <RickyAmes@aol.com>\nReceived: from 129.97.78.23 ([211.202.101.74])\n\tby speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;\n\tSun, 8 Apr 2007 13:07:21 -0400\nReceived: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100\nMessage-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>\nFrom: "Tomas Jacobs" <RickyAmes@aol.com>\nReply-To: "Tomas Jacobs" <RickyAmes@aol.com>\nTo: the00@speedy.uwaterloo.ca\nSubject: Generic Cialis, branded quality@ \nDate: Sun, 08 Apr 2007 21:00:48 +0300\nX-Mailer: Microsoft Outlook Express 6.00.2600.0000\nMIME-Version: 1.0\nContent-Type: multipart/alternative;\n\tboundary="--8896484051606557286"\nX-Priority: 3\nX-MSMail-Priority: Normal\nStatus: RO\nContent-Length: 988\nLines: 24\n\n----8896484051606557286\nContent-Type: text/html;\nContent-Transfer-Encoding: 7Bit\n\n<html>\n<body bgcolor="#ffffff">\n<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0

In [13]:
# Parsear el primer correo
mail, label = parse_email(index[0])
print("El correo es: ",label, "\n")
print(mail)

El correo es:  spam 

{'subject': ['gener', 'ciali', 'brand', 'qualiti'], 'body': ['do', 'feel', 'pressur', 'perform', 'rise', 'occas', 'tri', 'viagra', 'anxieti', 'thing', 'past', 'back', 'old', 'self'], 'content_type': 'multipart/alternative'}


El algoritmo de regresión Logística no es capaz de ingerir texto como parte del DataSet. Por lo tanto deben aplicarse una seri de funciones adicionales que transformen el texto de los correos electrónicos parseados en una representación númerica.

### Aplicación de CountVectorizer

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

# Preparacion del email en una cadena de texto.
prep_email = [" ".join(mail['subject']) + " ".join(mail['body'])]

vectorizer = CountVectorizer()
X = vectorizer.fit(prep_email)

print("e-mail:", prep_email, "\n")
print("Caracteristicas de entrada:", vectorizer.get_feature_names_out())

e-mail: ['gener ciali brand qualitido feel pressur perform rise occas tri viagra anxieti thing past back old self'] 

Caracteristicas de entrada: ['anxieti' 'back' 'brand' 'ciali' 'feel' 'gener' 'occas' 'old' 'past'
 'perform' 'pressur' 'qualitido' 'rise' 'self' 'thing' 'tri' 'viagra']


In [15]:
X = vectorizer.transform(prep_email)
print("\nValues:\n", X.toarray())


Values:
 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


#### Aplicación de OneHotEncoding

In [16]:
from sklearn.preprocessing import OneHotEncoder

prep_email = [[w] for w in mail['subject'] + mail ['body']]
enc = OneHotEncoder(handle_unknown = 'ignore')
X = enc.fit_transform(prep_email)

print("Features:\n", enc.get_feature_names_out(), "\n")
print("\nValues:\n", X.toarray())

Features:
 ['x0_anxieti' 'x0_back' 'x0_brand' 'x0_ciali' 'x0_do' 'x0_feel' 'x0_gener'
 'x0_occas' 'x0_old' 'x0_past' 'x0_perform' 'x0_pressur' 'x0_qualiti'
 'x0_rise' 'x0_self' 'x0_thing' 'x0_tri' 'x0_viagra'] 


Values:
 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

#### Funciones auxiliares para el preprocesamiento del DataSet

In [17]:
def create_prep_dataset(index_path, n_elements):
    X = []
    y = []
    indexes = parse_index(index_path, n_elements)
    for i in range(n_elements):
        print("\rParsing email: {0}".format(i+1), end = '')
        mail, label = parse_email(indexes[i])
        X.append(" ".join(['subject']) + " ".join(mail['body']))
        y.append(label)
    return X, y

In [18]:
# Leer unicamente un sunconjunto de 1000 correos electronicos
x_train, y_train = create_prep_dataset("/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/full/index", 1000)
x_train

Parsing email: 5

Parsing email: 6

Parsing email: 8

Parsing email: 12

Parsing email: 16

...

Parsing email: 953

Parsing email: 969

Parsing email: 991

Parsing email: 1000

['subjectdo feel pressur perform rise occas tri viagra anxieti thing past back old self',
 'subjecthi ive updat gulu i check mirror it seem littl typo debianreadm file exampl httpgulususherbrookecadebianreadm ftpftpfrdebianorgdebianreadm test lenni access releas diststest the current test develop snapshot name etch packag test unstabl pass autom test propog releas etch replac lenni like readmehtml yan morin consult en logiciel libr yanmorinsavoirfairelinuxcom 5149941556 to unsubscrib email debianmirrorsrequestlistsdebianorg subject unsubscrib troubl contact listmasterlistsdebianorg',
 'subjectmega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click httpwwwmoujsjkhchumcom authent viagra mega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click',
 'subjecthey billi realli fun go night talk said felt insecur manhood i notic toilet quit small area worri websit i tell secret weapon extra 3 inch trust girl love bigger one ive 5 time mani chi

 Aplicar vectorizacion a los datos

In [19]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)

In [20]:
print(x_train.toarray())
print("\n Features", len(vectorizer.get_feature_names_out())) 

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

 Features 20207


In [21]:
import pandas as pd

pd.DataFrame(x_train.toarray(), columns=[vectorizer.get_feature_names_out()])

,00,000,0000,000000,000002,000048000000000,000099,0000ff,000115000000000,0001171749,...,绰tel,绰۹ϵͳctsƽ,肾ǝvă,鏗ėvłq,饻jwk,鵵χ,낢ȏglgwă,뼰ʱϵ,쫷ƹư,쵼ã
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
y_train

['spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'ham',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam'

#### Entrenamiento del algoritmo de regresion logistica con el DataSet preprocesado

In [23]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(x_train, y_train)

LogisticRegression()

<img src="http://localhost:5000/archivos/20.png">


#### 4.- Prediccion

In [24]:
# Lectura de un DataSet de correos nuevos

# Leer 1500 correos de nuestro DataSet y quedarnos unicamente con los 500 ultimos 
# correos electronicos los cuales no se han utilizado para entrenar el algoritmo.
x, y = create_prep_dataset("/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/full/index", 150)
x_test = x[100:]
y_test = y[100:]


Parsing email: 14

Parsing email: 31

Parsing email: 42

Parsing email: 55

Parsing email: 79

Parsing email: 90

Parsing email: 110

Parsing email: 121

Parsing email: 140

Parsing email: 150

#### Preprocesamiento de los correos electronicos con el vectorizado creado anteriormente

In [25]:
x_test = vectorizer.transform(x_test)

In [26]:
y_pred = clf.predict(x_test)
y_pred

array(['spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam',
       'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham',
       'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam'], dtype='<U4')

In [27]:
print("Prediccion\n", y_pred)
print("\nEtiquetas reales", y_test)

Prediccion
 ['spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'ham' 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'ham' 'spam' 'spam' 'ham' 'spam' 'spam' 'ham' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam']

Etiquetas reales ['spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam']


#### Evaluacion de resultados 

In [28]:
from sklearn.metrics import accuracy_score
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy: 1.000


# 5.- Aumentando el DataSet 

In [29]:
# Leer 20,000 correos electronicos 
x, y = create_prep_dataset("/home/lia/Escritorio/api/archivos/datasets////datasets/trec07p/full/index", 20000)

Parsing email: 1

Parsing email: 13

Parsing email: 27

Parsing email: 38

Parsing email: 45

...

Parsing email: 19942

Parsing email: 19958

Parsing email: 19973

Parsing email: 19995

Parsing email: 20000

In [30]:
# Utilizamos 15,000 para entrenar el algoritmo y 5000 para realizar pruebas 
x_train, y_train = x[:15000], y[:15000]
x_test, y_test = x[15000:], y[15000:]

In [31]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)

In [32]:
clf = LogisticRegression()
clf.fit(x_train, y_train)

LogisticRegression()

<img src="http://localhost:5000/archivos/21.png">


In [33]:
x_test = vectorizer.transform(x_test)
y_pred = clf.predict(x_test)

In [34]:
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy: 0.988
